In [15]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.getOrCreate()

22/03/24 19:16:41 WARN Utils: Your hostname, c189 resolves to a loopback address: 127.0.1.1; using 192.168.0.133 instead (on interface wlp0s20f3)
22/03/24 19:16:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/24 19:16:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [45]:
schema = "ORDER_ID string, USER_ID string, ORDER_NUMBER string, ORDER_DOW integer, ORDER_HOUR_OF_DAY integer, DAYS_SINCE_PRIOR_ORDER float, ORDER_DETAIL string"

In [51]:
df = spark.read.csv("sample-datasets/*.csv", header=True, schema=schema)

In [52]:
df.count()

96621

In [53]:
df = (
    df
    .withColumn(
        "ORDER_DETAIL"
        , F.split(F.col("ORDER_DETAIL"), "~")
    )
    .withColumn(
        "temp"
        , F.explode(F.col("ORDER_DETAIL"))
    )
    .withColumn(
        "product"
        , F.split("temp", "\\|")[0]
    )
    .withColumn(
        "aisles"
        , F.split("temp", "\\|")[1]
    )
    .withColumn(
        "sequential"
        , F.split("temp", "\\|")[2]
    )
    .drop("ORDER_DETAIL", "temp")
)

In [54]:
df.count()

1021748

In [55]:
df.show(10)

+--------+-------+------------+---------+-----------------+----------------------+--------------------+--------------------+----------+
|ORDER_ID|USER_ID|ORDER_NUMBER|ORDER_DOW|ORDER_HOUR_OF_DAY|DAYS_SINCE_PRIOR_ORDER|             product|              aisles|sequential|
+--------+-------+------------+---------+-----------------+----------------------+--------------------+--------------------+----------+
| 1000867| 198377|           5|        0|               14|                   9.0|Triscuit Baked Wh...|            crackers|         8|
| 1000867| 198377|           5|        0|               14|                   9.0|Nutter Butter Coo...|       cookies cakes|        10|
| 1000867| 198377|           5|        0|               14|                   9.0|    Chili With Beans|  canned meals beans|         6|
| 1000867| 198377|           5|        0|               14|                   9.0|Zingers Raspberry...|       cookies cakes|         6|
| 1000867| 198377|           5|        0|       

In [68]:
spark.sql("""
    with temp as (select
        ORDER_ID
        , product
        , rank() over(partition by ORDER_ID, product order by sequential) rnk
    from t1)
    select *
    from temp
    where rnk <> 1
""").show(10)

+--------+-------+---+
|ORDER_ID|product|rnk|
+--------+-------+---+
+--------+-------+---+



In [56]:
df.createOrReplaceTempView("t1")

# The comparison between the aisles with the most products versus the best-selling product

In [73]:
spark.sql("""
    with product_order as (
        select
            distinct
            product
            , aisles
        from t1
    )
    select
        aisles
        , count(product) tot_products
    from product_order
    group by aisles
    order by tot_products desc
    limit 10
""").show()

+-------------------+------------+
|             aisles|tot_products|
+-------------------+------------+
|             yogurt|         889|
|    candy chocolate|         875|
|      ice cream ice|         857|
|     chips pretzels|         816|
|            missing|         800|
|    packaged cheese|         776|
|       frozen meals|         724|
|energy granola bars|         668|
|                tea|         632|
|      juice nectars|         617|
+-------------------+------------+



In [71]:
spark.sql("""
    with product_order as (
        select
            distinct
            ORDER_ID
            , product
            , aisles
        from t1
    )
    select
        product
        , aisles
        , count(*) sails
    from product_order
    group by product, aisles
    order by sails desc
    limit 10
""").show(10, False)

+----------------------+--------------------------+-----+
|product               |aisles                    |sails|
+----------------------+--------------------------+-----+
|Banana                |fresh fruits              |14022|
|Bag of Organic Bananas|fresh fruits              |11553|
|Organic Strawberries  |fresh fruits              |8156 |
|Organic Baby Spinach  |packaged vegetables fruits|7349 |
|Large Lemon           |fresh fruits              |6030 |
|Organic Avocado       |fresh fruits              |5605 |
|Organic Hass Avocado  |fresh fruits              |5451 |
|Strawberries          |fresh fruits              |4857 |
|Limes                 |fresh fruits              |4481 |
|Organic Raspberries   |packaged vegetables fruits|4094 |
+----------------------+--------------------------+-----+



# The top ten best selling products

In [70]:
spark.sql("""
    with product_order as (
        select
            distinct
            ORDER_ID
            , product
        from t1
    )
    select
        product
        , count(*) sails
    from product_order
    group by product
    order by sails desc
    limit 10
""").show(10, False)

+----------------------+-----+
|product               |sails|
+----------------------+-----+
|Banana                |14022|
|Bag of Organic Bananas|11553|
|Organic Strawberries  |8156 |
|Organic Baby Spinach  |7349 |
|Large Lemon           |6030 |
|Organic Avocado       |5605 |
|Organic Hass Avocado  |5451 |
|Strawberries          |4857 |
|Limes                 |4481 |
|Organic Raspberries   |4094 |
+----------------------+-----+



# The days of the week with the highest number of orders processed

In [74]:
spark.sql("""
    with orders_week_day as (
        select
            distinct
            ORDER_ID
            , ORDER_DOW
        from t1
    )
    select
        ORDER_DOW
        , count(*) total_per_day
    from orders_week_day
    group by ORDER_DOW
    order by total_per_day desc
""").show()

+---------+-------------+
|ORDER_DOW|total_per_day|
+---------+-------------+
|        0|        27465|
|        1|        19672|
|        5|        17401|
|        2|        16103|
|        4|        15959|
|     null|           21|
+---------+-------------+



# View the number of orders per day and hour

In [63]:
spark.sql("""
    with orders_week_day as (
        select
            distinct
            ORDER_ID
            , ORDER_DOW
            , ORDER_HOUR_OF_DAY
        from t1
        where
            True
            and ORDER_DOW is not null
            and ORDER_HOUR_OF_DAY is not null
    )
    select
        ORDER_DOW
        , ORDER_HOUR_OF_DAY
        , count(*) total_of_orders
    from orders_week_day
    group by ORDER_DOW, ORDER_HOUR_OF_DAY
    order by ORDER_DOW, ORDER_HOUR_OF_DAY desc
""").show()

+---------+-----------------+---------------+
|ORDER_DOW|ORDER_HOUR_OF_DAY|total_of_orders|
+---------+-----------------+---------------+
|        0|               24|              1|
|        0|               23|            315|
|        0|               22|            505|
|        0|               21|            650|
|        0|               20|            811|
|        0|               19|           1042|
|        0|               18|           1408|
|        0|               17|           1800|
|        0|               16|           2243|
|        0|               15|           2518|
|        0|               14|           2593|
|        0|               13|           2417|
|        0|               12|           2415|
|        0|               11|           2354|
|        0|               10|           2245|
|        0|                9|           1774|
|        0|                8|           1222|
|        0|                7|            567|
|        0|                6|     